# 概述
经典数解法    
干道协调，对称放行，一个7个路口，ABCDEFG    
间距：300，250，380，450，300，210    
绿信比：0.5, 0.55, 0.6, 0.6, 0.5, 0.7, 0.55    
公共信号周期范围：40-120s    
速度：12m/s (43.2km/h)  

In [1]:
import pandas as pd
import numpy as np
import math

# parameters
node_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
spacing = [300, 250, 380, 450, 300, 210]
ratio = [0.5, 0.55, 0.6, 0.6, 0.5, 0.7, 0.55]    # green time ratio
c = 80
step = 1    # idea distance will iterate with step
v = 12    # unit: m/s
M = 200

In [2]:
a = round(0.5 * v * c)
idea_dis_min = a - M 
idea_dis_max = a + M
print("idea distance in: {} to {}".format(idea_dis_min, idea_dis_max))

idea distance in: 280 to 680


# 最佳间距及周期

In [3]:
cols = ['idea_dis'] + node_list + ['b', 'b_index']
dis_diff = pd.DataFrame(columns=cols)

dis = [0] + spacing
dis_pre = [0]
for i in range(1, len(dis)):
    dis_pre.append(dis_pre[-1] + dis[i])
print(dis_pre)

for i in range(idea_dis_min, idea_dis_max+step, step):
    tmp = []
    for j in dis_pre:
        tmp.append(j % i)
        
    #print(tmp)

    tmp_sorted = sorted(tmp)
    tmp_sorted.append(i)
    #print(tmp_sorted)

    tmp_diff = []
    for j in range(1, len(tmp_sorted)):
        tmp_diff.append(tmp_sorted[j] - tmp_sorted[j-1])
    #tmp_diff.append(tmp_sorted[0] - tmp_sorted[-1])
    #print(tmp_diff)

    diff_max = max(tmp_diff)
    diff_max_id = tmp_diff.index(diff_max)
    #print(diff_max)

    row = [i] + tmp + [diff_max, diff_max_id]
    #print(row)

    dis_diff.loc[len(dis_diff.index)] = row 

print(dis_diff)

dis_diff_max = dis_diff[dis_diff['b'] == dis_diff['b'].max()]
dis_diff_max = dis_diff_max.reset_index(drop=True)
print(dis_diff_max)

dis_opt = dis_diff_max.at[0, 'idea_dis']

c_opt = 2 * dis_opt / v
print(c_opt)

[0, 300, 550, 930, 1380, 1680, 1890]
    idea_dis  A    B    C    D    E    F    G    b b_index
0        280  0   20  270   90  260    0  210  120       3
1        281  0   19  269   87  256  275  204  117       2
2        282  0   18  268   84  252  270  198  114       2
3        283  0   17  267   81  248  265  192  111       2
4        284  0   16  266   78  244  260  186  108       2
..       ... ..  ...  ...  ...  ...  ...  ...  ...     ...
396      676  0  300  550  254   28  328  538  226       1
397      677  0  300  550  253   26  326  536  227       1
398      678  0  300  550  252   24  324  534  228       1
399      679  0  300  550  251   22  322  532  229       1
400      680  0  300  550  250   20  320  530  230       1

[401 rows x 10 columns]
  idea_dis  A    B    C    D   E    F    G    b b_index
0      680  0  300  550  250  20  320  530  230       1
113.33333333333333


# 实际交叉口对应理想交叉口的编号和方位

In [4]:
dis_diff_opt = dis_diff_max.loc[0]
print(dis_diff_opt)

ideas_count = int(np.ceil(dis_pre[-1] / dis_diff_opt['idea_dis']) + 1)
print("count: {}".format(ideas_count))

id_reals_to_ideas = []    # the id of ideas matched to reals
pos_reals_to_ideas = []    # 0 means real is on the left, 1 means right
dis_reals_to_ideas = []

list_dis_diff_opt = dis_diff_opt[1:-2]
print(list_dis_diff_opt)
list_dis_diff_opt = list(list_dis_diff_opt.values)
list_dis_diff_opt.sort()

x = (dis_diff_opt['idea_dis'] - dis_diff_opt['b']) / 2
y = list_dis_diff_opt[dis_diff_opt['b_index']]
offset = x - y    # offset for dis_pre
dis_pre_offseted = [i + offset for i in dis_pre]
#print("offseted: {}".format(dis_pre_offseted))

for i in range(len(dis_pre_offseted)):
    dis_real_to_ideas = []
    dis_abs = []

    for j in range(ideas_count):
        tmp = dis_pre_offseted[i] - dis_diff_opt['idea_dis']*j
        dis_real_to_ideas.append(tmp)
        dis_abs.append(np.abs(tmp))
    #print(dis_real_to_ideas)

    dis_min = np.min(dis_abs)
    dis_min_index = dis_abs.index(dis_min)
    #print("id: {}, val: {}".format(dis_min_index, dis_min))

    id_reals_to_ideas.append(dis_min_index)
    dis_reals_to_ideas.append(dis_real_to_ideas[dis_min_index])

    if dis_real_to_ideas[dis_min_index] < 0:
        pos_reals_to_ideas.append(0)
    else:
        pos_reals_to_ideas.append(1)

print(id_reals_to_ideas)
print(pos_reals_to_ideas)
print(dis_reals_to_ideas)

idea_dis    680
A             0
B           300
C           550
D           250
E            20
F           320
G           530
b           230
b_index       1
Name: 0, dtype: object
count: 4
A      0
B    300
C    550
D    250
E     20
F    320
G    530
Name: 0, dtype: object
[0, 1, 1, 2, 2, 3, 3]
[1, 0, 1, 0, 1, 0, 1]
[205.0, -175.0, 75.0, -225.0, 225.0, -155.0, 55.0]


# 损失，相位差

In [5]:
# lost green time ratio
loss = [np.abs(i / dis_diff_opt['idea_dis']) for i in dis_reals_to_ideas]
loss

phase_diff = []
for i in range(len(id_reals_to_ideas)):
    if id_reals_to_ideas[i] % 2 == 0:
        phase_diff.append(0.5 - 0.5*ratio[i])
    else:
        phase_diff.append(1 - 0.5*ratio[i])

valid = []    # valid green time ratio
for i in range(len(ratio)):
    valid.append(ratio[i] - loss[i])

valid_left = []
valid_right= []
for i in range(len(pos_reals_to_ideas)):
    if pos_reals_to_ideas[i] == 0:
        valid_left.append(valid[i])
    else:
        valid_right.append(valid[i])

width = (min(valid_left) + min(valid_right)) * 0.5

print("loss: {}".format(loss))
print("phase different: {}".format(phase_diff))
print("valid ratio: {}".format(valid))
print("width: {}".format(width))

loss: [0.3014705882352941, 0.25735294117647056, 0.11029411764705882, 0.33088235294117646, 0.33088235294117646, 0.22794117647058823, 0.08088235294117647]
phase different: [0.25, 0.725, 0.7, 0.2, 0.25, 0.65, 0.725]
valid ratio: [0.1985294117647059, 0.2926470588235295, 0.48970588235294116, 0.2691176470588235, 0.16911764705882354, 0.47205882352941175, 0.4691176470588236]
width: 0.21911764705882353


# 结果展示

In [6]:
idx = ['ideal intersection id', 'position', 'green time ratio', 'phase diffrernce', 'lost ratio', 'valid ratio']

pos_char = []
for i in pos_reals_to_ideas:
    if i == 0:
        pos_char.append('left')
    else:
        pos_char.append('right')

print('Optimal cycle: {}'.format(c_opt))
print('Band width: {}'.format(width))
output_table = [id_reals_to_ideas, pos_char, ratio, phase_diff, loss, valid]
df_output = pd.DataFrame(output_table, columns=node_list, index=idx)
df_output

Optimal cycle: 113.33333333333333
Band width: 0.21911764705882353


,A,B,C,D,E,F,G
ideal intersection id,0,1,1,2,2,3,3
position,right,left,right,left,right,left,right
green time ratio,0.5,0.55,0.6,0.6,0.5,0.7,0.55
phase diffrernce,0.25,0.725,0.7,0.2,0.25,0.65,0.725
lost ratio,0.301471,0.257353,0.110294,0.330882,0.330882,0.227941,0.080882
valid ratio,0.198529,0.292647,0.489706,0.269118,0.169118,0.472059,0.469118
